In [1]:
#bank csv for kmeans
import math
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the data
Data = pd.read_table("C:\\Users\\Tamanna khan\\Downloads\\bank_full.csv", sep=None, engine="python")

# Select columns to drop and encode
cols = ["age", "balance", "day", "duration", "campaign", "pdays", "previous"]
data_encode = Data.drop(cols, axis=1)
data_encode = data_encode.apply(LabelEncoder().fit_transform)
data_rest = Data[cols]

# Concatenate the data
Data = pd.concat([data_rest, data_encode], axis=1)

# Splitting data into training and testing
data_train, data_test = train_test_split(Data, test_size=0.5, random_state=4)
X_train = data_train.drop("y", axis=1)
Y_train = data_train["y"]
X_test = data_test.drop("y", axis=1)
Y_test = data_test["y"]

# Scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Apply KMeans clustering
K_cent = 8
km = KMeans(n_clusters=K_cent, max_iter=100)
km.fit(X_train)
cent = km.cluster_centers_

# Calculate sigma
max_dist = 0
for i in range(K_cent):
    for j in range(K_cent):
        d = np.linalg.norm(cent[i] - cent[j])
        if d > max_dist:
            max_dist = d
sigma = max_dist / math.sqrt(2 * K_cent)

# Create Gaussian matrix
shape = X_train.shape
row = shape[0]
column = K_cent
G = np.empty((row, column), dtype=float)
for i in range(row):
    for j in range(column):
        dist = np.linalg.norm(X_train[i] - cent[j])
        G[i][j] = math.exp(-math.pow(dist, 2) / math.pow(2 * sigma, 2))

# Calculate weights
GTG = np.dot(G.T, G)
GTG_inv = np.linalg.inv(GTG)
fac = np.dot(GTG_inv, G.T)
W = np.dot(fac, Y_train)

# Create Gaussian matrix for test data
shape = X_test.shape
row = shape[0]
column = K_cent
G_test = np.empty((row, column), dtype=float)
for i in range(row):
    for j in range(column):
        dist = np.linalg.norm(X_test[i] - cent[j])
        G_test[i][j] = math.exp(-math.pow(dist, 2) / math.pow(2 * sigma, 2))

# Predict and calculate accuracy
prediction = np.dot(G_test, W)
prediction = 0.5 * (np.sign(prediction - 0.5) + 1)
score = accuracy_score(prediction, Y_test)
print(score.mean())


ModuleNotFoundError: No module named 'sklearn'